In [2]:
from pathlib import Path
from mri_data import utils
import nibabel as nib
import os
import json
import re
import statistics
import numpy as np
import pyperclip
import subprocess
from mri_data import file_manager as fm
import pandas as pd

In [14]:
drive_root = fm.get_drive_root()
dataroot = drive_root / "3Tpioneer_bids"
training_work_home = Path("/home/srs-9/Projects/prl_project/training_work_dirs")
# training_work_home = drive_root / "srs-9/prl_project/training_work_dirs"
work_dir = training_work_home / "test_train0_swinunet" #! edit this
ensemble_dir = work_dir / "ensemble_output"
datalist_file = work_dir / "datalist.json"

prl_info = pd.read_csv("/home/srs-9/Projects/prl_project/data/PRL_labels.csv", index_col="subid")

def subid_from_subject(subject):
    return int(re.search(r"(\d{4})", subject)[1])
# prl_info['subid'] = prl_info['ID'].map(subid_from_subject)
# prl_info.set_index("subid", inplace=True)

In [18]:
with open(datalist_file, 'r') as f:
    datalist = json.load(f)

inferences = []
labels = []
subjects = []
subids = []
for item in datalist['testing']:
    label = str(drive_root / Path(item['label']).relative_to("/media/smbshare"))
    assert os.path.exists(label)
    labels.append(label)
    subject = re.search(r"(sub-ms(\d{4}))", label)[1]
    subjects.append(subject)
    subids.append(int(re.search(r"(sub-ms(\d{4}))", label)[2]))
    session = re.search(r"(ses-\d{8})", label)[1]
    inf = ensemble_dir / subject / session / "flair.phase.t1.nii.gz"
    assert inf.exists()
    inferences.append(str(inf))

In [31]:

data = {"subid": [], "lesion_dice": [], "prl_dice": [], "nPRL": []}
for sub, lab, inf in zip(subids, labels, inferences):
    lab_data = nib.load(lab).get_fdata()
    inf_data = nib.load(inf).get_fdata()
    lesion_dice = utils.dice_score(lab_data, inf_data, seg1_val=1, seg2_val=1)
    prl_dice = utils.dice_score(lab_data, inf_data, seg1_val=2, seg2_val=2)

    data['subid'].append(sub)
    data['prl_dice'].append(prl_dice)
    data['lesion_dice'].append(lesion_dice)
    data['nPRL'].append(prl_info.loc[sub, "Total PRL"])

df_pred = pd.DataFrame(data)
df_pred.set_index("subid", inplace=True)
df_pred = df_pred[['nPRL', 'prl_dice', 'lesion_dice']]

In [29]:
print("lesion dice: {:0.2f}".format(df_pred['lesion_dice'].mean()))
print("PRL    dice: {:0.2f}".format(df_pred['prl_dice'].mean()))

lesion dice: 0.81
PRL    dice: 0.53


In [32]:
df_pred

,nPRL,prl_dice,lesion_dice
subid,,,
1183,4,0.734107,0.817032
1281,7,0.281852,0.766164
2115,1,0.057955,0.835972
1136,2,0.000000,0.800117
1523,2,0.461511,0.795537
1274,2,0.131425,0.870389
2131,1,0.915875,0.897268
1152,2,0.635897,0.795990
2207,1,0.000000,0.729639


In [42]:
subid = 1136
print(f"PRL's: {prl_info.loc[subid, "Total PRL"]}")

ind = subjects.index(f"sub-ms{subid}")
# ind = 5
ground_truth = labels[ind]
inf = inferences[ind]
image_dir = Path(ground_truth).parent.parent
lesion_dir = Path(ground_truth).parent

additional_labels = []
additional_labels = [lesion_dir / lab for lab in additional_labels]

images = ["flair.nii.gz", "phase.nii.gz", "t1.nii.gz"]
image_paths = [image_dir / im for im in images]

label_paths = [inf, ground_truth] + additional_labels

cmd = utils.open_itksnap_workspace_cmd(image_paths, label_paths, win=False)
# print(cmd)
pyperclip.copy(cmd)
subprocess.Popen(cmd.split(" "))

PRL's: 2


<Popen: returncode: None args: ['itksnap', '-g', '/media/smbshare/3Tpioneer_...>

In [34]:
def get_performance2(lab, inf):
    TP = np.sum((lab == 2) & (inf == 2))
    FP = np.sum((lab == 1) & (inf == 2))
    TN = np.sum((lab == 1) & (inf == 1))
    FN = np.sum((lab == 2) & (inf == 1))
    return TP, FP, TN, FN

In [35]:
TP_total = 0
FP_total = 0
TN_total = 0
FN_total = 0
for lab, inf in zip(labels, inferences):
    lab_data = nib.load(lab).get_fdata()
    inf_data = nib.load(inf).get_fdata()
    TP, FP, TN, FN = get_performance2(lab_data, inf_data)
    TP_total += TP
    FP_total += FP
    TN_total += TN
    FN_total += FN

When there is a PRL, it has a 50% chance of identifying it. When it's not a PRL, it will most likely identify it correctly. So it's better than chance overall

In [36]:
sens = TP_total / (TP_total + FN_total)
spec = TN_total / (TN_total + FP_total)
ppv = TP_total / (TP_total + FP_total)
npv = TN_total / (TN_total + FN_total)

print("Sensitivity: {:0.4f}".format(sens))
print("Specificity: {:0.4f}".format(spec))
print("PPV:         {:0.4f}".format(ppv))
print("NPV:         {:0.4f}".format(npv))

Sensitivity: 0.5253
Specificity: 0.9629
PPV:         0.6354
NPV:         0.9429
